In [33]:
!rm -rf camp_data
!git clone https://github.com/HongJeSeong/camp_data.git

Cloning into 'camp_data'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 111 (delta 40), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (111/111), 1.82 MiB | 5.10 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [34]:
!ls camp_data

a.csv		covidTime.csv  influenzaCovid.csv  review.txt  weather_mod.csv
covidKorea.csv	img	       README.md	   src	       wine.csv


In [35]:
import pandas as pd
df = pd.read_csv("camp_data/review.txt", names=['sentence', 'label'], sep='\t')
df

,sentence,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0


In [63]:
from sklearn.model_selection import train_test_split
x = df['sentence'].values
y = df['label'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=20)

데이터를 읽어 들이고
단어 빈도에 따른 사용할 단어 개수의 최대값. 가장 빈번하게 사용되는 num_words개의 단어만 저장

In [72]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test) 
vocab_size=len(tokenizer.word_index)+1

In [ ]:
tokenizer.word_index

In [74]:
x_train[5], xtrain[5]

("I don't each much pasta, but I love the homemade /hand made pastas and thin pizzas here.",
 [3, 64, 325, 106, 22, 3, 79, 1, 83, 2, 66])

![padding](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/padding.PNG)
(https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/padding.PNG)

In [75]:
xtrain=pad_sequences(xtrain,padding='post', maxlen=20)
xtest=pad_sequences(xtest,padding='post', maxlen=20)

In [76]:
xtrain[1]

array([228,  66,  50,  10,  58, 477, 122,  59,  89,   7, 252,  68,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=20) )
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

![d](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/dropout.PNG)
(https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/dropout.PNG)
학습의 오버피팅을 줄이기 위한 레이어

In [ ]:
history = model.fit(xtrain,y_train, epochs=120, verbose=1,validation_data=(xtest,y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 10))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [91]:
import numpy as np
def pred_result(text,model):
  text= tokenizer.texts_to_sequences(text)
  text=pad_sequences(text,padding='post', maxlen=20)
  pred = model.predict(text)

  print("확률 : ",pred)
  if pred > 0.5:
    print("Positive")
  else:
    print("Negative")


In [92]:
pred_result(["great"],model)

확률 :  [[0.99611855]]
Positive
